In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np
import time
import csv
import os
import time
from datetime import datetime
from neo4j import GraphDatabase

# Neo4j configuration
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "Shady5000$")

In [ ]:

class UAVNetworkSimulation:

    def __init__(self, uri, auth):
        self.driver = GraphDatabase.driver(uri, auth=auth)

    def close(self):
        self.driver.close()
    
    def create_initial_graph(self, num_uavs, connection_range, ground_station_pos, backbone_range):
        #Create undirected graph
        G = nx.Graph()

        # Define typical throughput and latency ranges (for example purposes)
        typical_throughput = (50, 100)  # Mbps
        typical_latency = (10, 50)     # Milliseconds
        typical_battery = (100,100) # %

        # Add UAV nodes with properties
        for i in range(num_uavs):
            throughput = random.randint(*typical_throughput)
            latency = random.randint(*typical_latency)
            battery = random.randint(*typical_battery)
            G.add_node(i, pos=(random.randint(0, 100), random.randint(0, 100)),
                       throughput=throughput, latency=latency, battery=battery)

        # Add The Backbone UAV
        backbone_uav_id = num_uavs
        G.add_node(backbone_uav_id, pos=(random.randint(0, 100), random.randint(0, 100)), is_backbone=True,
                       throughput=throughput, latency=latency, battery=battery)

        # Connect Backbone UAV to other UAVs Based on Euclidean Distance
        for i in range(num_uavs):
            pos_i = G.nodes[i]['pos']
            pos_backbone = G.nodes[backbone_uav_id]['pos']
            distance = np.linalg.norm(np.array(pos_i) - np.array(pos_backbone))
            if distance <= backbone_range:
                G.add_edge(i, backbone_uav_id, weight=distance)

        # Add and connect the Ground Station To The Backbone Only
        ground_station_id = num_uavs + 1
        G.add_node(ground_station_id, pos=ground_station_pos, is_ground_station=True, throughput=throughput, latency=latency, battery=battery)
        G.add_edge(backbone_uav_id, ground_station_id)

        # Add edges between UAVs within connection range
        for i in range(num_uavs):
            for j in range(i + 1, num_uavs):
                pos_i = G.nodes[i]['pos']
                pos_j = G.nodes[j]['pos']
                distance = np.linalg.norm(np.array(pos_i) - np.array(pos_j))
                if distance <= connection_range:
                    G.add_edge(i, j, weight=distance)

        return G
    
    # Upload Current Graph To Neo4j
    def upload_to_neo4j(self, G):
        with self.driver.session() as session:
            # Clear existing data
            session.run("MATCH (n) DETACH DELETE n")

            # Add nodes
            for node in G.nodes:
                is_backbone = G.nodes[node].get('is_backbone', False)
                is_ground_station = G.nodes[node].get('is_ground_station', False)
                is_attack_node = G.nodes[node].get('is_attack_node', False)

                if is_backbone:
                    session.run(f"""
                        CREATE (b:BackboneUAV {{id: $id, pos: $pos, uavType: $uavType,
                            throughput: $throughput, latency: $latency, battery: $battery  + '%'}})
                        """, id=node, pos=G.nodes[node]['pos'], uavType = "Backbone UAV", throughput=G.nodes[node]['throughput'],
                        latency=G.nodes[node]['latency'], battery=G.nodes[node]['battery'])
                elif is_ground_station:
                    session.run(f"""
                        CREATE (g:GroundStation {{id: $id, pos: $pos, gsName: $GS, battery: $battery + '%'}})
                        """, id=node, pos=G.nodes[node]['pos'], GS = "Ground Station", battery=G.nodes[node]['battery'])
                else:
                    session.run(f"""
                        CREATE (u:UAV {{id: $id, pos: $pos, uavType: $uavType, uavName: $uavName,
                            throughput: $throughput + 'mb/s', latency: $latency + 'ms', battery: $battery + '%'}})
                        """, id=node, pos=G.nodes[node]['pos'], uavType = "UAV", uavName = "UAV"  + str(node), throughput=G.nodes[node]['throughput'],
                        latency=G.nodes[node]['latency'], battery=G.nodes[node]['battery'])

            # Add relationships
            for source, target in G.edges:
                session.run("""
                    MATCH (a), (b)
                    WHERE a.id = $source AND b.id = $target
                    CREATE (a)-[:COMMUNICATES_WITH]->(b)
                    """, source=source, target=target)

    # Draw Graph
    def draw_graph(self, G):
        pos = nx.get_node_attributes(G, 'pos')

        # Labelling Each Type Of Node
        labels = {}
        for node in G.nodes:
            if G.nodes[node].get('is_backbone', False):
                labels[node] = 'Backbone UAV'
            elif G.nodes[node].get('is_ground_station', False):
                labels[node] = 'Ground Station'
            elif G.nodes[node].get('is_attack_node', False):
                labels[node] = 'Attack Node'
            else:
                labels[node] = f'UAV{node}'

        # Differentiate node types
        regular_nodes = [node for node in G.nodes if not G.nodes[node].get('is_backbone', False) and not G.nodes[node].get('is_ground_station', False)]
        backbone_nodes = [node for node in G.nodes if G.nodes[node].get('is_backbone', False)]
        ground_station_nodes = [node for node in G.nodes if G.nodes[node].get('is_ground_station', False)]
        attacking_nodes = [node for node in G.nodes if G.nodes[node].get('is_attack_node', False)]

        # Draw regular UAV nodes
        nx.draw_networkx_nodes(G, pos, nodelist=regular_nodes, node_color='green', node_size=300, label='UAV')
        
        # Draw backbone UAV node
        nx.draw_networkx_nodes(G, pos, nodelist=backbone_nodes, node_color='lightgreen', node_size=500, label='Backbone UAV')

        # Draw ground station node
        nx.draw_networkx_nodes(G, pos, nodelist=ground_station_nodes, node_color='lightblue', node_size=500, label='Ground Station')

        nx.draw_networkx_nodes(G, pos, nodelist=attacking_nodes, node_color='red', node_size=500, label='Attack Node')

        # Draw edges
        nx.draw_networkx_edges(G, pos, edge_color='gray')

        # Draw custom labels
        nx.draw_networkx_labels(G, pos, labels)

        plt.title("Multilayer UAV Network")
        plt.legend()
        plt.show()
        
    def update_uav_positions(self, G, move_range):
        
        # Update the positions of UAVs randomly within a given range.
    
        for node in G.nodes:
            if 'is_backbone' not in G.nodes[node] and 'is_ground_station' not in G.nodes[node]:
                current_pos = G.nodes[node]['pos']
                new_pos = (current_pos[0] + random.uniform(-move_range, move_range), 
                           current_pos[1] + random.uniform(-move_range, move_range))
                G.nodes[node]['pos'] = new_pos
        return G

    def update_network_connections(self, G, connection_range, backbone_range):
        
        # Update the network connections based on new positions and connection range.
        # Excluding backbone UAV and ground station
        num_uavs = len(G.nodes) - 2  
        backbone_uav_id = num_uavs

        # Update connections for regular UAVs
        for i in range(num_uavs):
            for j in range(i + 1, num_uavs):
                if G.has_edge(i, j):
                    G.remove_edge(i, j)
                pos_i = G.nodes[i]['pos']
                pos_j = G.nodes[j]['pos']
                distance = np.linalg.norm(np.array(pos_i) - np.array(pos_j))
                if distance <= connection_range:
                    G.add_edge(i, j, weight=distance)

        # Update connections for the Backbone UAV
        for i in range(num_uavs):
            if G.has_edge(i, backbone_uav_id):
                G.remove_edge(i, backbone_uav_id)
            pos_i = G.nodes[i]['pos']
            pos_backbone = G.nodes[backbone_uav_id]['pos']
            distance = np.linalg.norm(np.array(pos_i) - np.array(pos_backbone))
            if distance <= backbone_range:
                G.add_edge(i, backbone_uav_id, weight=distance)

        return G

    def update_neo4j_database(self, G):
        
        # Update the Neo4j database with the new positions and connections.
        
        with self.driver.session() as session:
            # Update positions
            for node in G.nodes:
                pos = G.nodes[node]['pos']
                session.run("""
                    MATCH (n)
                    WHERE n.id = $id
                    SET n.pos = $pos
                    """, id=node, pos=pos)

            # Update relationships
            session.run("MATCH (n)-[r:COMMUNICATES_WITH]->() DELETE r")
            for source, target in G.edges:
                session.run("""
                    MATCH (a), (b)
                    WHERE a.id = $source AND b.id = $target
                    CREATE (a)-[:COMMUNICATES_WITH]->(b)
                    """, source=source, target=target)
    
    def generate_network_traffic(self, G, num_packets):
        
        # Generate network traffic considering the throughput of UAVs.
        
        packets = []
        for _ in range(num_packets):
            # Exclude backbone and ground station
            source = random.randint(0, len(G.nodes) - 3)  
            target = random.randint(0, len(G.nodes) - 3)
            
            # Packet size should not exceed the throughput of the source UAV
            max_packet_size = G.nodes[source]['throughput']
            size = random.randint(1, max_packet_size)

            packets.append({'source': source, 'target': target, 'size': size})
        return packets
    
    def route_packets(self, G, packets):
        
        # Determine the path for each packet and calculate cumulative latency.
        
        for packet in packets:
            source = packet['source']
            target = packet['target']
            if nx.has_path(G, source, target):
                path = nx.shortest_path(G, source, target)
                packet['path'] = path
                packet['delivered'] = True

                # Calculate cumulative latency
                total_latency = sum(G.nodes[node]['latency'] for node in path)
                packet['total_latency'] = total_latency
            else:
                packet['path'] = []
                packet['delivered'] = False
                packet['total_latency'] = None
        return packets


    def add_attack_node(self, G, target_ids, num_uavs, attack_node_pos=(random.randint(0, 100), random.randint(0, 100))):
        
        # Calculate a new ID for the attack node by taking the maximum current ID + 1
        
        attack_node_id = max(G.nodes()) + 1
        attk_battery = (100,100)
        battery = random.randint(*attk_battery)
        
        # Add the attack node with specified properties
        G.add_node(attack_node_id, pos=attack_node_pos, nodeName="Attack Node", is_attack_node=True, throughput=1000, latency=1, battery=battery)

        # Directly connect the attack node to the target node
        for target_id in target_ids:
            if target_id in G.nodes():
                G.add_edge(attack_node_id, target_id, weight=5)  # Weight can be adjusted or calculated based on actual metrics

        return attack_node_id

    def upload_attack_node_to_neo4j(self, G, attack_node_id, target_ids):
        
        with self.driver.session() as session:
            # Retrieve node data from the graph
            attack_node = G.nodes[attack_node_id]
            
            # Create the attack node in Neo4j
            session.run("""
                CREATE (a:AttackNode {
                    id: $id, 
                    pos: $pos,
                    nodeName: $nodeName,
                    throughput: $throughput, 
                    latency: $latency, 
                    battery: $battery, 
                    is_attack_node: $is_attack_node
                })
                """, id=attack_node_id, 
                    pos=str(attack_node['pos']),
                    nodeName=attack_node['nodeName'],
                    throughput=attack_node['throughput'], 
                    latency=attack_node['latency'], 
                    battery=attack_node['battery'], 
                    is_attack_node=True)

            # Create a relationship from the attack node to the target UAV
            for target_id in target_ids:
                if target_id in G.nodes():
                    session.run("""
                        MATCH (a:AttackNode {id: $attack_id}), (u:UAV {id: $target_id})
                        CREATE (a)-[:ATTACKS]->(u)
                        """, attack_id=attack_node_id, target_id=target_id)

    def update_network_metrics(self, G):
        
        for node in G.nodes():
            # Randomly increase latency by up to 10ms and decrease throughput by up to 10 Mbps
            G.nodes[node]['latency'] = max(1, G.nodes[node]['latency'] + random.randint(-5, 5))
            G.nodes[node]['throughput'] = max(1, G.nodes[node]['throughput'] + random.randint(-10, 10))

    def update_network_ddos_metrics(self, G):
        
        for node in G.nodes():
            # Randomly increase latency by up to 10ms and decrease throughput by up to 10 Mbps
            G.nodes[node]['latency'] += random.randint(1, 5)
            G.nodes[node]['throughput'] = max(1, G.nodes[node]['throughput'] - random.randint(1, 10))

    def run_simulation(self, G, total_time, update_interval, move_range, connection_range, backbone_range, num_packets):
    
        # Run the simulation for a specified period of time.
        results = []    
        start_time = time.time()
        iteration = 0
        while time.time() - start_time < total_time:
            print(f"Iteration {iteration}:")
            
            # Update UAV positions
            self.update_uav_positions(G, move_range)

            # Update network connections
            self.update_network_connections(G, connection_range, backbone_range)
            self.update_network_metrics(G) 

            # Update Neo4j database and redraw the graph
            # self.update_neo4j_database(G)
            # self.draw_graph(G)

            packets = self.generate_network_traffic(G, num_packets)
            routed_packets = self.route_packets(G, packets)
            # Capture and record metrics for each node
            current_time = time.time() - start_time
            for node_id in G.nodes():
                results.append({
                    'time': current_time,
                    'node_id': node_id,
                    'latency': G.nodes[node_id]['latency'],
                    'throughput': G.nodes[node_id]['throughput']
                })
            print(results)

            # Log the routed packets information
            for i, packet in enumerate(routed_packets):
                print(f"  Packet {i}: Source: {packet['source']}, Target: {packet['target']}, "
                      f"Size: {packet['size']}, Delivered: {packet['delivered']}, "
                      f"Path: {packet['path']}, Total Latency: {packet['total_latency']}")

            # Wait for the next update
            time.sleep(update_interval)
            iteration += 1
        return results

    def simulate_ddos_attack(self, G, attack_node_id, target_ids, increase_factor, decrease_factor):
        for target_id in target_ids:
            if target_id in G.nodes():
                G.nodes[target_id]['latency'] = int(G.nodes[target_id]['latency'] * increase_factor)
                G.nodes[target_id]['throughput'] = int(G.nodes[target_id]['throughput'] * decrease_factor)
                G.nodes[target_id]['battery'] = int(G.nodes[target_id]['battery'] * decrease_factor)
                # Apply effects to neighboring nodes, ensuring integer operations
                for neighbor in G.neighbors(target_id):
                    G.nodes[neighbor]['latency'] = int(G.nodes[neighbor]['latency'] * (increase_factor - 0.1))
                    G.nodes[neighbor]['throughput'] = int(G.nodes[neighbor]['throughput'] * (decrease_factor + 0.1))
                    # G.nodes[neighbor]['battery'] = int(G.nodes[neighbor]['battery'] * (decrease_factor + 0.1))
                    if 'battery' in G.nodes[neighbor]:
                        G.nodes[neighbor]['battery'] = int(G.nodes[neighbor]['battery'] * (decrease_factor + 0.1))


    def run_ddos_simulation(self, G, total_time, update_interval, move_range, connection_range, backbone_range, num_packets, attack_node_id, target_ids):
        metrics = []  # List to store metrics over time
        start_time = time.time()
        iteration = 0
        
        # Define the directory and filename
        directory = "simulation_results"
        if not os.path.exists(directory):
            os.makedirs(directory)
        filename = f"{directory}/ddos_simulation_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        
        while time.time() - start_time < total_time:
            print(f"Iteration {iteration}:")
            
            # Simulation updates
            self.update_uav_positions(G, move_range)
            self.update_network_connections(G, connection_range, backbone_range)
            self.update_network_metrics(G)
            
            # Trigger DDoS attack
            if iteration == 1:
                self.simulate_ddos_attack(G, attack_node_id, target_ids, 0.5, 0.5)
            
            # Traffic generation and routing
            packets = self.generate_network_traffic(G, num_packets)
            routed_packets = self.route_packets(G, packets)
            
            # Capture and log metrics
            current_metrics = [
                (node, time.time() - start_time, G.nodes[node]['latency'], G.nodes[node]['throughput'], G.nodes[node]['battery'])
                for node in G.nodes()
            ]
            metrics.extend(current_metrics)
            
            # Print packet info
            for i, packet in enumerate(routed_packets):
                print(f"Packet {i}: Source: {packet['source']}, Target: {packet['target']}, Size: {packet['size']}, Delivered: {packet['delivered']}, Path: {packet['path']}, Total Latency: {packet['total_latency']}")
            
            # Delay for next update
            time.sleep(update_interval)
            iteration += 1
        
        # Save metrics to CSV
        with open(filename, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Node', 'Time', 'Latency', 'Throughput', 'Battery'])
            writer.writerows(metrics)
        
        print(f"Metrics saved to {filename}")
        return metrics

    def plot_time_series(self, results):
        import matplotlib.pyplot as plt

        # Filter results for a specific node for clarity, e.g., node 0
        node_results = [res for res in results if res['node_id'] == 0]
        times = [res['time'] for res in node_results]
        latencies = [res['latency'] for res in node_results]
        throughputs = [res['throughput'] for res in node_results]

        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.plot(times, latencies, marker='o', color='r')
        plt.title('Latency Over Time for Node 0')
        plt.xlabel('Time (s)')
        plt.ylabel('Latency (ms)')

        plt.subplot(1, 2, 2)
        plt.plot(times, throughputs, marker='o', color='b')
        plt.title('Throughput Over Time for Node 0')
        plt.xlabel('Time (s)')
        plt.ylabel('Throughput (Mbps)')

        plt.tight_layout()
        plt.show()
    
    def analyze_network(self, G):
        # Calculate basic properties
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        sparsity = nx.density(G)  # Density close to 0 indicates a sparse graph
        
        # Centrality measures
        degree_centrality = nx.degree_centrality(G)  # Normalized to (n-1)
        closeness_centrality = nx.closeness_centrality(G)
        betweenness_centrality = nx.betweenness_centrality(G)
        
        # Identifying the most central node
        most_central_node = max(degree_centrality, key=degree_centrality.get)
        
        # Network robustness (using node connectivity)
        connectivity = nx.node_connectivity(G)
        
        # Plotting
        pos = nx.spring_layout(G)  # positions for all nodes
        plt.figure(figsize=(8, 8))
        nx.draw_networkx_nodes(G, pos, node_size=700)
        nx.draw_networkx_edges(G, pos, width=6)
        nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")
        plt.axis("off")
        plt.show()

        return {
            "Number of Nodes": num_nodes,
            "Number of Edges": num_edges,
            "Sparsity": sparsity,
            "Degree Centrality": degree_centrality,
            "Closeness Centrality": closeness_centrality,
            "Betweenness Centrality": betweenness_centrality,
            "Most Central Node": most_central_node,
            "Connectivity": connectivity
        }
    


Here we create the original graph and assign standard through put and latency values to each UAV. More nodes can be added to the network by parsing a new value to the "num_uavs" parameters below. Connections between nodes are based on node range and Euclidean distance.  

In [ ]:
"""Creating The Network First"""
# Simulation Parameters
num_uavs = 10
connection_range = 30
ground_station_pos = (50, 50)
backbone_range = 50
num_packets = 10

total_time = 60  # Total time to run the simulation in seconds
update_interval = 5  # Time interval between updates in seconds
move_range = 5

# Initialize Simulation
simulation = UAVNetworkSimulation(URI, AUTH)

# Create and Setup UAV Network
uav_network = simulation.create_initial_graph(num_uavs, connection_range, ground_station_pos, backbone_range)

# Upload to Neo4j
simulation.upload_to_neo4j(uav_network)

# Draw Initial Graph
simulation.draw_graph(uav_network)

# Close Neo4j Connection
simulation.close()

We run our movement simulation code, in the process UAVs can make new communications as they are currently routing fictional packets. We log the results below immediately and generate data based on our routing logic. Perform descriptive statistics.

In [ ]:
"""Routing Simulation"""
# Run the Simulation
results = simulation.run_simulation(uav_network, total_time, update_interval, move_range, connection_range, backbone_range, num_packets)

# Update network connections
uav_network = simulation.update_network_connections(uav_network, connection_range, backbone_range)

# Update Neo4j Database with the new positions and connections
simulation.update_neo4j_database(uav_network)

# Now draw the graph to visualize the updated network
simulation.draw_graph(uav_network)

# Close Neo4j Connection
simulation.close()

simulation.plot_time_series(results)

Adding The Attack Node.

In [ ]:
#Add Attack Node
# Initialize the simulation object and create the initial UAV network
simulation = UAVNetworkSimulation(URI, AUTH)
uav_network = simulation.update_network_connections(uav_network, connection_range, backbone_range)

# # Upload the initial UAV network to Neo4j
simulation.upload_to_neo4j(uav_network)

# Target UAV ID for the DDoS attack
target_uav_ids = [5, 7] # Example: Target UAV ID that you want to attack

# Add the attack node and connect it to the target UAV
attack_node_id = simulation.add_attack_node(uav_network, target_uav_ids, num_uavs)
simulation.upload_attack_node_to_neo4j(uav_network, attack_node_id, target_uav_ids)

# Optionally, visualize the updated network to confirm the addition and connection of the attack node
simulation.draw_graph(uav_network)

# Close the Neo4j connection at the end of the session
simulation.close()


In [ ]:
"""DDoS Simulation"""
# Initialize your network G here
target_nodes = [5, 7]  # Target node IDs that will be affected by the DDoS attack
# Assuming you have an attack node and some target nodes
attack_node_id = simulation.add_attack_node(uav_network, target_nodes, num_uavs)  # ID of your attack node

total_time = 120  # For example, run the simulation for 120 seconds
update_interval = 5
move_range = 10
connection_range = 30
backbone_range = 50
num_packets = 10
target_ids = [5, 7]  # Targeting nodes 5 and 7 for the DDoS attack

# Run the DDoS simulation
simulation.run_ddos_simulation(uav_network, total_time, update_interval, move_range, connection_range, backbone_range, num_packets, attack_node_id, target_ids)


Descriptive Statistics, Performing Correlation Analysis On the Dataset and plotting correlation matrix.

In [ ]:
import matplotlib.pyplot as plt
metrics = simulation.run_ddos_simulation(uav_network, total_time, update_interval, move_range, connection_range, backbone_range, num_packets, attack_node_id, target_ids)

def plot_network_metrics(metrics):
    # Assuming metrics is a list of tuples (node_id, time, latency, throughput, battery)
    # Filter by node_id for node 5 and node 7
    times_5 = [m[1] for m in metrics if m[0] == 5]
    latencies_5 = [m[2] for m in metrics if m[0] == 5]
    throughputs_5 = [m[3] for m in metrics if m[0] == 5]

    times_7 = [m[1] for m in metrics if m[0] == 7]
    latencies_7 = [m[2] for m in metrics if m[0] == 7]
    throughputs_7 = [m[3] for m in metrics if m[0] == 7]

    plt.figure(figsize=(12, 6))
    
    plt.subplot(2, 1, 1)
    plt.plot(times_5, latencies_5, label='Latency UAV 5')
    plt.plot(times_7, latencies_7, label='Latency UAV 7')
    plt.title('Latency over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Latency (ms)')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(times_5, throughputs_5, label='Throughput UAV 5')
    plt.plot(times_7, throughputs_7, label='Throughput UAV 7')
    plt.title('Throughput over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Throughput (Mbps)')
    plt.legend()

    plt.tight_layout()
    plt.show()
    
if metrics:
    plot_network_metrics(metrics)
else:
    print("No data was returned from the simulation.")
# Example usage:
# metrics = [(5, 1.0, 50, 100), (7, 1.0, 70, 90), (5, 2.0, 55, 95), (7, 2.0, 75, 85)]
# plot_network_metrics(metrics)


In [ ]:
"""Pearson's Correlation"""

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Data provided
data = [
    {'time': 0.001983642578125, 'node_id': 0, 'latency': 16, 'throughput': 71},
    {'time': 0.001983642578125, 'node_id': 1, 'latency': 41, 'throughput': 54},
    {'time': 0.001983642578125, 'node_id': 2, 'latency': 32, 'throughput': 76},
    {'time': 0.001983642578125, 'node_id': 3, 'latency': 7, 'throughput': 52},
    {'time': 0.001983642578125, 'node_id': 4, 'latency': 18, 'throughput': 53},
    {'time': 0.001983642578125, 'node_id': 5, 'latency': 17, 'throughput': 65},
    {'time': 0.001983642578125, 'node_id': 6, 'latency': 16, 'throughput': 84},
    {'time': 0.001983642578125, 'node_id': 7, 'latency': 40, 'throughput': 55},
    {'time': 0.001983642578125, 'node_id': 8, 'latency': 37, 'throughput': 68},
    {'time': 0.001983642578125, 'node_id': 9, 'latency': 39, 'throughput': 77},
    {'time': 0.001983642578125, 'node_id': 10, 'latency': 36, 'throughput': 86},
    {'time': 0.001983642578125, 'node_id': 11, 'latency': 39, 'throughput': 94},
    {'time': 5.015164613723755, 'node_id': 0, 'latency': 17, 'throughput': 78},
    # Assuming the rest of the data points are appended here in the same format...
]

# Converting data into a pandas DataFrame
df = pd.DataFrame(data)

# Calculating the correlation between latency and throughput
correlation_matrix = df[['latency', 'throughput']].corr()
print(correlation_matrix)


# Plotting a heatmap of the correlation matrix
plt.figure(figsize=(8, 6))
heatmap = sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
heatmap.set_title('Correlation Heatmap between Latency and Throughput', pad=12)
plt.show()


In [ ]:
def plot_network_metrics_from_csv(file_path):
    # Load data from CSV
    df = pd.read_csv(file_path)
    
    # Filter data for specific nodes
    node_5_df = df[df['Node'] == 5]
    node_7_df = df[df['Node'] == 7]
    
    # Extract times, latencies, and throughputs
    times_5 = node_5_df['Time']
    latencies_5 = node_5_df['Latency']
    throughputs_5 = node_5_df['Throughput']

    times_7 = node_7_df['Time']
    latencies_7 = node_7_df['Latency']
    throughputs_7 = node_7_df['Throughput']

    plt.figure(figsize=(12, 6))
    
    plt.subplot(2, 1, 1)
    plt.plot(times_5, latencies_5, label='Latency UAV 5')
    plt.plot(times_7, latencies_7, label='Latency UAV 7')
    plt.title('Latency over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Latency (ms)')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(times_5, throughputs_5, label='Throughput UAV 5')
    plt.plot(times_7, throughputs_7, label='Throughput UAV 7')
    plt.title('Throughput over Time')
    plt.xlabel('Time (s)')
    plt.ylabel('Throughput (Mbps)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Example usage:
file_path = 'simulation_results\ddos_simulation_20240423_012712.csv'
plot_network_metrics_from_csv(file_path)


In [ ]:
simulation.analyze_network(uav_network)

In [24]:
from pymongo import MongoClient

def get_database():
    # Provide the mongodb atlas url to connect python to mongodb using pymongo
    CONNECTION_STRING = "mongodb://localhost:27017"
    
    # Create a connection using MongoClient
    client = MongoClient(CONNECTION_STRING)

    print("Connecting to MongoDB...")
    db = client.your_database
    collection = db.your_collection
    print("Connected to MongoDB.")

    print("Inserting data...")
    collection.insert_one({"key": "value"})
    print("Data inserted.")

db = get_database()


Connecting to MongoDB...
Connected to MongoDB.
Inserting data...
Data inserted.
